# 영상의 특징 추출

- edge는 pixel 값이 급격히 변화하는 지점이다.
- 따라서 pixel을 미분하여 미분 값이 큰 부분을 edge로 검출하게 된다.
- **Sobel**에서는 X 방향 edge 검출과 Y 방향 edge 검출을 위해 별도의 커널을 사용한다.
  - Sobel x는 수직선 방향의 edge를 검출한다.
![image](https://user-images.githubusercontent.com/61646760/124232128-38b02f00-db4c-11eb-9281-2a8208b534b2.png)
  - Sobel y는 수평선 방향의 edge를 검출한다.
![image](https://user-images.githubusercontent.com/61646760/124232157-42d22d80-db4c-11eb-85f4-21b5d9d1655a.png)

In [ ]:
# cv2.Sobel(src, ddepth, dx, dy, dst, ksize, scale, delta, borderType) -> dst

# src : 입력영상
# ddepth : 출력영상의 데이터 타입 (-1)
# dx : x 방향 미분차수
# dy : x 방향 미분차수
# dst : 출력영상
# ksize : 커널의 크기
# scale : 연산결과에 추가적으로 곱할 값
# delta : 연산결과에 추가적으로 더할 값
# borderType : 가장자리 픽셀확장 방식

In [3]:
import sys
import numpy as np
import cv2


src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

kernel = np.array([[-1, 0, 1], 
                   [-2, 0, 2], 
                   [-1, 0, 1]], dtype=np.float32)  # X 방향 edge 검출을 위한 커널

# print(kernel)

# [[-1.  0.  1.]
#  [-2.  0.  2.]
#  [-1.  0.  1.]]

# dx = cv2.filter2D(src, -2, kernel, delta = 128)  # kernel을 직접 만들어 2D로 filtering

# Sobel() : 3개의 pixel로 계산해서 값이 커지면 edge로 판단 (gradient)
dx = cv2.Sobel(src, -1, 1, 0, delta=128)  # x 방향 미분 (1, 0)
dy = cv2.Sobel(src, -1, 0, 1, delta=128)  # y 방향 미분 (0, 1)

cv2.imshow('src', src)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)
cv2.waitKey()

cv2.destroyAllWindows()

![image](https://user-images.githubusercontent.com/61646760/124230538-174e4380-db4a-11eb-973e-f1223e4bf0fa.png)
- dx는 좌측 기둥이 잡히는데, dy는 잡히지 않는다.

## Sobel edge

In [ ]:
# magnitude(x, y, magnitude) -> magnitude

$$magnitude = \sqrt{dx^2+dy^2}$$

In [5]:
# x, y를 모두 고려해서 edge 구하기
import sys
import numpy as np
import cv2


src = cv2.imread('lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

dx = cv2.Sobel(src, cv2.CV_32F, 1, 0)  # X 방향 1차 미분 값, type을 임의로 정해 둠(CV_32F)
dy = cv2.Sobel(src, cv2.CV_32F, 0, 1)  # Y 방향 1차 미분 값

mag = cv2.magnitude(dx, dy)                  # magnitude 구하기
mag = np.clip(mag, 0, 255).astype(np.uint8)  # 범위 clipping (mag에 대해 0보다 작으면 0으로, 255보다 크면 255로 변경)

dst = np.zeros(mag.shape[:2], np.uint8)  # 0으로 채우기
dst[mag > 120] = 255                     # 120보다 크면 255(1)로 변경
# _, dst = cv2.threshold(mag, 120, 255, cv2.THRESH_BINARY)

cv2.imshow('src', src)
cv2.imshow('mag', mag)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()

![image](https://user-images.githubusercontent.com/61646760/124236194-21277500-db51-11eb-8844-c7111c2b64e7.png)

## Canny edge 검출

In [ ]:
#Canny(image, threshold1, threshold2, edges, apertureSize, L2gradient) -> edges
# image : 입력 영상
# threshold1: 에지결정 하한값
# threshold1: 에지결정 상한값
# edges: None
# apertureSize: 커널사이즈
# L2gradient: gradient 크기 계산, False

In [6]:
import sys
import numpy as np
import cv2


src = cv2.imread('building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

dst = cv2.Canny(src, 50, 150)  # edge 결정 하한값 50, 상한값 150

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()

![image](https://user-images.githubusercontent.com/61646760/124237847-f1796c80-db52-11eb-960b-e3a068ec93dd.png)

## 허프변환 : 직선검출

In [ ]:
# HoughLinesP(image, rho, theta, threshold, lines, minLineLength, maxLineGap) -> lines
# image: 입력 에지영상
# rho: 축적배열에서 rho의 간격
# theta: 축적배열에서 theta의 간격
# threshold: 직선판단할 임계값
# lines: 선분의 끝좌표 (x1, y1, x2, y2)
# srn = None, stn = None
# minLineLength: 검출한 선분의 최소 길이
# maxLineGap: 직선으로 간주할 최대 에지 점 간격 (끝어진 점을 연결할 기준)

In [9]:
import sys
import numpy as np
import cv2


src = cv2.imread('building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

edges = cv2.Canny(src, 50, 150)

lines = cv2.HoughLinesP(edges, 1, np.pi / 180., 160,
                        minLineLength=50, maxLineGap=5)

dst = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

if lines is not None:
    for i in range(lines.shape[0]):
        pt1 = (lines[i][0][0], lines[i][0][1])  # 시작점 좌표
        pt2 = (lines[i][0][2], lines[i][0][3])  # 끝점 좌표
        cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()